In [3]:
import sqlite3
from sqlite3 import Error

In [27]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn, query):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    query_1 = """select name, total_revenue 
from (select  f.name, sum(case when b.memid <> 0 then f.membercost * b.slots else f.guestcost * b.slots end) as total_revenue 
from Bookings b 
join Facilities f on b.facid = f.facid 
group by f.name
     )tab 
where total_revenue < 1000
order by 2"""
    
    query_2 = """select 
  s2.member_name, 
  concat(s2.recommender_firstname,',',s2.recommender_surname) as recommender_name 
from 
  (
    select 
      s1.member_name, 
      s1.recommender_id as member_id, 
      m.firstname as recommender_firstname, 
      m.surname as recommender_surname 
    from 
      (
        select 
          m2.memid as member_id, 
          concat(m1.firstname,' ',m1.surname) as member_name,
          m2.recommendedby as recommender_id 
        from 
          Members as m1 
          inner join Members as m2 on m1.memid = m2.memid 
        where 
          (
            m2.recommendedby is not null 
            or m2.recommendedby <> ' ' 
            or m2.recommendedby <> ''
          ) 
          and m1.memid <> 0
      ) as s1 
      left join Members as m on s1.recommender_id = m.memid 
    WHERE 
      m.memid <> 0
  ) as s2
"""

    query_3 = """
    select 
  f.name as facility, 
  sum(b.slots) as slot 
from 
  Bookings as b 
  left join Facilities as f on f.facid = b.facid 
  left join Members as m on m.memid = b.memid 
where 
  b.memid <> 0 
group by
  facility 
order by 
  slot desc;
    """
    
    query_4 = """select 
  s.month, 
  s.fname AS facility, 
  sum(s.snumber) AS slot 
from 
  (
    select 
      strftime('%m', starttime) as month, 
      f.name AS fname, 
      b.slots AS snumber 
    from 
      Bookings as b 
      left join Facilities as f on f.facid = b.facid 
      left join Members as m on m.memid = b.memid 
    where 
      b.memid <> 0
  ) s 
group by
  month, 
  facility 
order by
  month, 
  slot desc"""
    
    
    with conn: 
        print("2. Query all tasks")
        print("""Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that theres a different cost for guests and members!""")
        select_all_tasks(conn, query_1)
        #print("""Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order""")
        #select_all_tasks(conn,query_2 )
        print("""Q12: Find the facilities with their usage by member, but not guests""")
        select_all_tasks(conn,query_3 )
        print("""Q13: Find the facilities usage by month, but not guests""")
        select_all_tasks(conn,query_4 )
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that theres a different cost for guests and members!
('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)
Q12: Find the facilities with their usage by member, but not guests
('Badminton Court', 1086)
('Tennis Court 1', 957)
('Massage Room 1', 884)
('Tennis Court 2', 882)
('Snooker Table', 860)
('Pool Table', 856)
('Table Tennis', 794)
('Squash Court', 418)
('Massage Room 2', 54)
Q13: Find the facilities usage by month, but not guests
('07', 'Tennis Court 1', 201)
('07', 'Massage Room 1', 166)
('07', 'Badminton Court', 165)
('07', 'Snooker Table', 140)
('07', 'Tennis Court 2', 123)
('07', 'Pool Table', 110)
('07', 'Table Tennis', 98)
('07', 'Squash Court', 50)
('07', 'Massage Room 2', 8)
('08', 'Badminton Court', 414)
('08', 'Tennis Court 2', 345)
('08', 'Tennis Court 1', 339)
('08', 'Snooker Table', 316)
(

Produce a report of members and who recommended them in alphabetic surname,firstname order 

In [ ]:
#I wasnt able to run the Q11 part since i got a error saying concat was not supported

select 
  s2.memberName as member_name, 
  concat(s2.recommenderfirstname,',',s2.recommendersurname) as recommender_name 
from 
  (
    select 
      s1.memberName as member_name, 
      s1.recommenderId as member_id, 
      m.firstname as recommender_firstname, 
      m.surname as recommender_surname 
    from 
      (
        select 
          m2.memid as member_id, 
          concat(m1.firstname,',',m1.surname) as member_name
          m2.recommendedby as recommender_id 
        from 
          Members as m1 
          inner join Members as m2 on m1.memid = m2.memid 
        where 
          (
            m2.recommendedby is not null 
            or m2.recommendedby <> ' ' 
            or m2.recommendedby <> ''
          ) 
          and m1.memid <> 0
      ) as s1 
      left join Members as m on s1.recommenderId = m.memid 
    WHERE 
      m.memid <> 0
  ) as s2;